In [1]:
import os

In [5]:
import pandas as pd

def build_forecast_prompt(forecast_df, context="retail", buffer_pct=0.5):
    mean_val = forecast_df["yhat"].mean()
    max_val  = forecast_df["yhat"].max()
    min_val  = forecast_df["yhat"].min()
    start    = pd.to_datetime(forecast_df["ds"].min()).strftime("%b %Y")
    end      = pd.to_datetime(forecast_df["ds"].max()).strftime("%b %Y")
    n_months = len(forecast_df)

    prompt = f"""
You are a financial analyst.

Context: {context} industry.
Forecast horizon: {start}–{end} ({n_months} months)
Avg inflation: {mean_val:.2f}%, Peak: {max_val:.2f}%, Low: {min_val:.2f}%, Buffer: {buffer_pct:.1f}%.

Write a concise summary (under 120 words) that includes:
1. One-line inflation trend.
2. Budget/pricing advice for management.
3. 1–2 sentence rationale with numbers.
4. One example memo line for internal reports.
    """
    return prompt.strip()

In [4]:
future_forecast = pd.read_csv('future_forecast.csv')

In [6]:
future_forecast

,ds,yhat,yhat_lower,yhat_upper
0,2024-02-29,1.010781,-1.517809,3.642086
1,2024-03-31,1.281863,-1.236236,3.893104
2,2024-04-30,-1.598486,-4.030598,0.907098
3,2024-05-31,-0.185697,-2.881283,2.232000
4,2024-06-30,2.544789,0.082684,5.176887
5,2024-07-31,3.687775,1.045202,6.239970
6,2024-08-31,2.791909,0.394032,5.142596
7,2024-09-30,2.113550,-0.521916,4.801766
8,2024-10-31,1.972912,-0.777832,4.513735
9,2024-11-30,0.652662,-1.970433,3.181800


In [7]:
!pip install transformers accelerate sentencepiece


In [8]:
from huggingface_hub import login

# Paste your token when prompted (don’t include quotes)
login()


In [9]:
from transformers import pipeline

generator = pipeline(
    "text-generation",
    model="distilgpt2",
    device_map="auto"
)

prompt = """You are a financial analyst.
Context: manufacturing industry.
Forecast horizon: Feb 2024–Jan 2025 (12 months)
Avg inflation: 1.73%, Peak: 3.69%, Low: -1.60%, Buffer: 0.5%.

Write a short inflation outlook with:
1. One-sentence inflation trend.
2. Budget/pricing recommendation.
3. Brief rationale.
4. Example memo line."""


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
result = generator(prompt, max_new_tokens=120, temperature=0.7, top_p=0.9)
print(result[0]["generated_text"])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a financial analyst.
Context: manufacturing industry.
Forecast horizon: Feb 2024–Jan 2025 (12 months)
Avg inflation: 1.73%, Peak: 3.69%, Low: -1.60%, Buffer: 0.5%.

Write a short inflation outlook with:
1. One-sentence inflation trend.
2. Budget/pricing recommendation.
3. Brief rationale.
4. Example memo line.
5. Budget/pricing recommendation.
6. Example memo line.
7. Example memo line.
8. Example memo line.
9. Example memo line.
10. Example memo line.
11. Example memo line.
12. Example memo line.
13. Example memo line.
14. Example memo line.
15. Example memo line.
16. Example memo line.
17. Example memo line.
18. Example memo line.
19. Example memo line.
20. Example memo line.
21. Example memo line


In [ ]:
generator = pipeline(
    "text-generation",
    model="HuggingFaceH4/zephyr-7b-alpha",
    torch_dtype="auto",
    device_map="auto"
)

config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

...

model-00005-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
def local_llm_summary(forecast_df, context="retail", buffer_pct=0.5):
    import pandas as pd
    mean_val = forecast_df["yhat"].mean()
    max_val  = forecast_df["yhat"].max()
    min_val  = forecast_df["yhat"].min()
    start    = pd.to_datetime(forecast_df["ds"].min()).strftime("%b %Y")
    end      = pd.to_datetime(forecast_df["ds"].max()).strftime("%b %Y")
    n_months = len(forecast_df)

    prompt = f"""
      You are a financial analyst.
      Context: {context} industry.
      Forecast horizon: {start}–{end} ({n_months} months)
      Avg inflation: {mean_val:.2f}%, Peak: {max_val:.2f}%, Low: {min_val:.2f}%, Buffer: {buffer_pct:.1f}%.

      Write a concise inflation summary:
      - Trend
      - Budget/pricing advice
      - Rationale
      - Example memo line
      """
    result = generator(prompt, max_new_tokens=200, temperature=0.7, top_p=0.9)
    return result[0]["generated_text"]


In [ ]:
summary = local_llm_summary(future_forecast, context="manufacturing")
print(summary)



You are a financial analyst.
Context: manufacturing industry.
Forecast horizon: Feb 2024–Jan 2025 (12 months)
Avg inflation: 1.73%, Peak: 3.69%, Low: -1.60%, Buffer: 0.5%.

Write a concise inflation summary:
- Trend
- Budget/pricing advice
- Rationale
- Example memo line

Trend:
Inflation is expected to remain relatively stable over the forecast horizon, with an average inflation rate of 1.73%. However, there are some fluctuations in the data, with a peak of 3.69% and a low of -1.60%. These fluctuations should be taken into account when making budget and pricing decisions.

Budget/pricing advice:
Given the relatively stable inflation rate, it is recommended to set a budget that accounts for a 1.73% inflation rate. This will allow for some buffer, as the inflation rate may fluctuate above or below the average. When setting prices, it is also recommended to consider the peak inflation rate of 3.69% and adjust prices accordingly.

Rationale:
The rationale behind this advice is to ensure 

In [ ]:
context_guidance = {
    "manufacturing": "Focus on input costs, raw materials, energy, and supplier contract adjustments.",
    "retail": "Emphasize consumer pricing, product margins, and demand elasticity.",
    "logistics": "Highlight transportation costs, fuel price fluctuations, and operational budgeting.",
    "finance": "Discuss interest rates, loan margins, and inflation hedging strategies.",
    "technology": "Focus on wage inflation, hardware import costs, and service pricing.",
    "agriculture": "Emphasize fertilizer, fuel, and crop yield cost implications.",
    "hospitality": "Focus on food cost, labor wage inflation, and seasonal pricing."
}


In [ ]:
def local_llm_summary(forecast_df, context="manufacturing", buffer_pct=0.5):
    import pandas as pd
    from transformers import pipeline

    mean_val = forecast_df["yhat"].mean()
    max_val  = forecast_df["yhat"].max()
    min_val  = forecast_df["yhat"].min()
    start    = pd.to_datetime(forecast_df["ds"].min()).strftime("%b %Y")
    end      = pd.to_datetime(forecast_df["ds"].max()).strftime("%b %Y")
    n_months = len(forecast_df)

    # use context_guidance dictionary
    guidance = context_guidance.get(context.lower(), "Provide general inflation advice.")

    prompt = f"""
You are a senior business analyst preparing a brief inflation outlook for the {context} sector.

Forecast horizon: {start}–{end} ({n_months} months)
Average inflation: {mean_val:.2f}%
Peak inflation: {max_val:.2f}%
Lowest inflation: {min_val:.2f}%
Adjustment buffer: {buffer_pct:.1f}%

Industry focus: {guidance}

Write a concise professional summary (≤150 words) with:
1. One-sentence inflation trend.
2. Clear {context}-specific budgeting or pricing advice.
3. A short rationale referencing numeric data.
4. A one-line example memo for management.
Use a factual, confident tone.
"""

    result = generator(prompt, max_new_tokens=200, temperature=0.6, top_p=0.9)
    return result[0]["generated_text"]


In [ ]:
contexts = ["manufacturing", "retail", "logistics", "finance", "technology"]

for ctx in contexts:
    print(f"\n--- {ctx.upper()} ---\n")
    print(local_llm_summary(future_forecast, context=ctx, buffer_pct=0.5))



--- MANUFACTURING ---


You are a senior business analyst preparing a brief inflation outlook for the manufacturing sector.

Forecast horizon: Feb 2024–Jan 2025 (12 months)
Average inflation: 1.73%
Peak inflation: 3.69%
Lowest inflation: -1.60%
Adjustment buffer: 0.5%

Industry focus: Focus on input costs, raw materials, energy, and supplier contract adjustments.

Write a concise professional summary (≤150 words) with:
1. One-sentence inflation trend.
2. Clear manufacturing-specific budgeting or pricing advice.
3. A short rationale referencing numeric data.
4. A one-line example memo for management.
Use a factual, confident tone.

Example:

Inflation trend:

In the upcoming 12 months, the manufacturing sector is expected to experience a moderate inflation rate of 1.73% on average, with peak inflation of 3.69% and the lowest inflation rate of -1.60%.

Budgeting or pricing advice:

To mitigate the impact of inflation on manufacturing costs, companies should consider implementing price i